# Billards

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook untersuchen wir die Bewegung einer Punktmasse in zweidimensionalen Billards. Innerhalb des Billards bewegt sich die Punktmasse frei, während sie an der Berandung elastisch reflektiert wird. Zwischen aufeinanderfolgenden Reflexionen ist die Bewegung analytisch bekannt, so dass numerisch nur die Auftreffpunkte auf die Berandung und die Geschwindigkeitsvektoren nach den Reflexionen zu bestimmen sind. Konkret betrachten wir ein symmetriereduziertes quadratisches Billard und ein symmetriereduziertes Sinai-Billard.

### Importanweisungen

Aus dem `functools`-Modul der Python-Standardbibliothek wird die Funktion `partial` importiert, die es erlaubt, einige der Argumente einer Funktion vorab festzulegen. Konkret werden wir diese Möglichkeit im Zusammenhang mit den Funktionen `time_to_straight_line` und `v_refl_straight_line` nutzen.

Um die Umrandung der Billards zu definieren und darzustellen, verwenden wir `Path` und `PathPatch` aus den `matplotlib`-Modulen `path` bzw. `patches`.

In [ ]:
from functools import partial
from math import atan2, cos, hypot, pi, sin, sqrt
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch

plt.style.use("numphyspy.style")

## Quadratisches Billard

### Kollision mit einer geraden Linie

Da das quadratische Billard genauso wie seine symmetriereduzierte Version, also ein Dreiecksbillard, durch Geradenstücke begrenzt ist, ist im Rahmen der stroboskopischen Abbildung zu bestimmen, welche Zeit eine Punktmasse von einem Anfangsort $(x|y)$ mit dem Geschwindigkeitsvektor

$$\boldsymbol{v} = \begin{pmatrix}v_x\\ v_y\end{pmatrix}$$

bis zum Auftreffen auf eine gerade Berandung benötigt und welche Geschwindigkeit sie nach der elastischen Kollision haben wird. Die Zeit, die von `time_to_straight_line` bestimmt wird, wird später bei mehreren Berandungsstücken benötigt, um zu entscheiden, welche Kollision für die weitere Bewegung relevant ist. Der Ort der Kollision zusammen mit dem von `v_refl_straight_line` berechneten Geschwindigkeitsvektor bildet die Anfangsbedingung für den nächsten Schritt der stroboskopischen Abbildung.

Die gerade Linie wird durch einen Punkt $(x_0|y_0)$ auf der Geraden und den Vektor in Richtung der Geraden mit den Komponenten `dir_x` und `dir_y` beschrieben.

Die beiden Funktionen werden später auch im Zusammenhang mit dem Sinai-Billard verwendet.

In [2]:
def time_to_straight_line(x_0, y_0, dir_x, dir_y, x, y,
                          v_x, v_y):
    denominator = v_x*dir_y - v_y*dir_x
    if denominator != 0:
        numerator = (x_0-x)*dir_y - (y_0-y)*dir_x
        return numerator / denominator
    else:
        return float("inf")

def v_refl_straight_line(dir_x, dir_y, x, y, v_x, v_y):
    projection = v_x*dir_x + v_y*dir_y
    return (-v_x + 2*dir_x*projection,
            -v_y + 2*dir_y*projection)

### Definition der Geometrie des symmetriereduzierten quadratischen Billards

Das quadratische Billard kann aufgrund seiner Symmetrie auf ein rechtwinkliges, gleichschenkliges Dreieck reduziert werden. Die Geometrie dieses Billards beschreiben wir mit Hilfe eines Dictionaries, das die folgenden Einträge enthält:
- `time`: Dieser Eintrag verweist auf eine Liste, die für jede der drei geraden Berandungslinien des Dreiecks eine Funktion enthält, die die Zeit bis zum Auftreffen auf der Berandungslinie berechnet. Hierbei wird die `partial`-Funktion verwendet, um die Eigenschaften der Berandunglinie festzulegen. Beim Aufruf der Funktion in der Liste müssen dann nur noch der Ort und die Geschwindigkeit der Punktmasse übergeben werden.
- `v_refl`: Dieser Eintrag verweist auf eine Liste, die für jede der drei geraden
Berandungslinien des Dreiecks eine Funktion enthält, die den Geschwindigkeitsvektor nach der Reflexion berechnet. Auch hier wird wieder von der `partial`-Funktion Gebrauch gemacht.
- `boundary_path`: Dieser Eintrag beschreibt die Berandung, um sie später graphisch darstellen zu können.

In [ ]:
TRIANGLE = {"time":
            [partial(time_to_straight_line, 0, 0, 1, 0),
             partial(time_to_straight_line, 1, 0, 0, 1),
             partial(time_to_straight_line,
                     0, 0, sqrt(0.5), sqrt(0.5))
             ],
            "v_refl":
            [partial(v_refl_straight_line, 1, 0),
             partial(v_refl_straight_line, 0, 1),
             partial(v_refl_straight_line,
                     sqrt(0.5), sqrt(0.5))
             ],
            "boundary_path":
            Path([(0, 0), (1, 0), (1, 1), (0, 0)],
                 [Path.MOVETO, Path.LINETO,
                 Path.LINETO, Path.CLOSEPOLY]
                 )
            }

### Stroboskopische Abbildung

Die nachfolgende Generatorfunktion liefert ausgehend von einem Startpunkt und einem Geschwindigkeitsvektor auf Anfrage den nächsten Ort einer Kollision mit der Berandung sowie den Geschwindigkeitsvektor nach der Kollision.

Dazu wird für jedes Berandungsstück ermittelt, wann eine Kollision mit diesem stattfinden würde. Die tatsächlich stattfindende Kollision ist dann diejenige mit der kleinsten positiven Zeit. Startet man auf der Berandung, so ist immer eine der Zeiten gleich null. Da diese Zeit aufgrund von Rundungsfehlern einen kleinen positiven Wert annehmen kann, wird gefordert, dass die Zeit größer als eine kleine Schranke `eps` ist. Anschließend werden der Ort der Kollision und der Geschwindigkeitsvektor nach der Kolision berechnet und abschließend zusammen mit dem Index des Berandungsstücks übergeben.

Diese Generatorfunktion wird auch beim Sinai-Billard verwendet.

In [ ]:
def stroboscopic_map(billard_type, x, y, v_x, v_y, eps):
    time_funcs = billard_type["time"]
    while True:
        dt_min = float("inf")
        n_min = None
        for n_boundary, time_func in enumerate(time_funcs):
            dt = time_func(x, y, v_x, v_y)
            if eps < dt < dt_min:
                dt_min = dt
                n_min = n_boundary
        if n_min is None:
            raise ValueError("kein Wandkontakt gefunden")
        x = x + v_x * dt_min
        y = y + v_y * dt_min
        v_ref_func = billard_type["v_refl"][n_min]
        v_x, v_y = v_ref_func(x, y, v_x, v_y)
        yield (n_min, x, y, v_x, v_y)

### Berechnung der Trajektorie

Da beide hier betrachteten Billards einen horizontalen Rand bei $y=0$ besitzen, lassen wir die Punktmasse dort starten, so dass nur noch die $x$-Koordinate des Startpunkts festgelegt werden muss. Da der Betrag der Geschwindigkeit irrelevant ist, wird er auf 1 gesetzt, so dass sich der Geschwindigkeitsvektor durch den Winkel $\alpha$ relativ zur $x$-Achse angeben lässt. 

Mit Hilfe von `stroboscopic_map` wird ein Generator `contacts` erzeugt, der unter Verwendung von `next` in der Schleife nacheinander die Orte der nächsten Kollisionen liefert.

In [ ]:
def trajectory(billard_type, x, alpha, n_out):
    y = 0
    v_x, v_y = (cos(alpha), sin(alpha))
    eps = 1e-8
    x_values = [x]
    y_values = [y]
    contacts = stroboscopic_map(
        billard_type, x, y, v_x, v_y, eps)
    n = 0
    while n < n_out:
        n_boundary, x, y, v_x, v_y = next(contacts)
        x_values.append(x)
        y_values.append(y)
        n = n+1
    return x_values, y_values

### Definition der Bedienelemente

Um eine Kopplung zwischen den Bedienelementen in den verschiedenen Abschnitten dieses Jupyter-Notebooks zu vermeiden, wird das Dictionary, das die Schieberegler, definiert, innerhalb einer Funktion neu erzeugt. Die einstellbaren Parameter sind
- `x`: $x$-Koordinate des Startpunkts auf der horizontalen Berandung bei $y=0$
- `alpha`: Winkel des Geschwindigkeitsvektors relativ zur $x$-Achse
- `n_out`: Anzahl der Punkte (wird nur in `plot_result_triangle` und `plot_result_sinai` verwendet)
- `n_out_poincare`: Anzahl der Punkte (wird nur in `plot_result_poincare` verwendet)

In [ ]:
def get_widgets():
    widget_dict = {
        "x":
        widgets.FloatSlider(
            value=0.75, min=0.55, max=0.95, step=0.05,
            description="$x$"),
        "alpha":
        widgets.FloatSlider(
            value=0.6, min=0.05, max=3.10, step=0.05,
            description=r"$\alpha$"),
        "n_out":
        widgets.IntSlider(
            value=25, min=1, max=100, step=1,
            description=r"$n_\text{out}$"),
        "n_out_poincare":
            widgets.IntSlider(
            value=10000, min=10, max=20000, step=10,
            description=r"$n_\text{out}$")
    }
    return widget_dict

### Graphische Darstellung der Trajektorie

Es wird die berechnete Trajektorie innerhalb der Dreiecksberandung dargestellt. Um die Seitenverhältnisse korrekt darzustellen, wird `set_aspect("equal")` verwendet.

In [ ]:
@interact(**get_widgets())
def plot_result_triangle(x, alpha, n_out):
    x_values, y_values = trajectory(TRIANGLE, x, alpha,
                                    n_out)
    path = TRIANGLE["boundary_path"]
    pathpatch = PathPatch(path, facecolor="none")
    fig, ax = plt.subplots()
    ax.add_patch(pathpatch)
    ax.set_axis_off()
    ax.set_aspect("equal")
    ax.plot(x_values, y_values)

## Sinai-Billard

Das Sinai-Billard entsteht aus dem quadratischen Billard, indem man innerhalb des Quadrats eine zusätzliche kreisförmige Berandung einführt, die im Mittelpunkt des Quadrats zentriert wird. Die Punktmasse kann sich somit innerhalb der quadratischen Berandung, aber nur außerhalb der kreisförmigen Berandung bewegen.

Auch hier werden wir wieder eine symmetriereduzierte Version des Billards betrachten, die ein Achtel des Sinai-Billards darstellt.

### Kollision mit einem Kreis

Zusätzlich zu den bereits definierten Funktionen `time_to_straight_line` und `v_refl_straight_line` für gerade Berandungen benötigen wir für das Sinai-Billard noch die entsprechende Funktionen für die Kollision mit einer kreisförmigen Berandung. Um die neuen Funktionen `time_to_circle` und `v_refl_circle` allgemein benutzen zu können, sind sowohl der Radius als auch der Ursprung des Kreises frei wählbar.

Das Problem des Schnittes einer Geraden mit einem Kreis führt auf eine quadratische Gleichung. Wenn die zugehörige Diskrimante negativ ist, trifft die Punktmasse gar nicht auf den Kreis. In diesem Fall wird die Zeit bis zur Kollision auf
unendlich gesetzt. Da die Kollision mit dem Berandungsstück stattfindet, das die kürzeste Zeit liefert, hat das zur Folge, dass es sich in diesem Fall nicht um den Kreis handelt.

In [ ]:
def time_to_circle(x_0, y_0, r, x, y, v_x, v_y):
    a = v_x**2 + v_y**2
    b = (x-x_0)*v_x + (y-y_0)*v_y
    c = (x-x_0)**2 + (y-y_0)**2 - r**2
    diskriminante = b**2 - a*c
    if diskriminante > 0:
        return (-b-sqrt(diskriminante))/a
    else:
        return float("inf")

def v_refl_circle(x_0, y_0, r, x, y, v_x, v_y):
    dir_x = -(y-y_0)/r
    dir_y = (x-x_0)/r
    return v_refl_straight_line(dir_x, dir_y, x, y,
                                v_x, v_y)

### Definition der Geometrie des symmetriereduzierten Sinai-Billards

Wie schon beim quadratischen Billard wird die Geometrie des symmetriereduzierten Sinai-Billards in einem Dictionary definiert. Neben drei geraden Berandungsstücken kommt hier noch ein kreisförmiges Berandungsstück hinzu.

In [ ]:
SINAI = {"time":
         [partial(time_to_straight_line, 0, 0, 1, 0),
          partial(time_to_straight_line, 1, 0, 0, 1),
          partial(time_to_straight_line, 0, 0,
                  sqrt(0.5), sqrt(0.5)),
          partial(time_to_circle, 0, 0, 0.5)
          ],
         "v_refl":
         [partial(v_refl_straight_line, 1, 0),
          partial(v_refl_straight_line, 0, 1),
          partial(v_refl_straight_line,
                  sqrt(0.5), sqrt(0.5)),
          partial(v_refl_circle, 0, 0, 0.5)
          ],
         "boundary_path":
         Path([*(0.5*Path.arc(0, 45).vertices),
               (1, 1), (1, 0), (0, 0)],
              [*Path.arc(0, 45).codes, Path.LINETO,
               Path.LINETO, Path.CLOSEPOLY]
              )
         }

### Graphische Darstellung der Trajektorie

In [ ]:
@interact(**get_widgets())
def plot_result_sinai(x, alpha, n_out):
    x_values, y_values = trajectory(SINAI, x, alpha, n_out)
    path = SINAI["boundary_path"]
    pathpatch = PathPatch(path, facecolor="none")
    fig, ax = plt.subplots()
    ax.add_patch(pathpatch)
    ax.set_axis_off()
    ax.set_aspect("equal")
    ax.plot(x_values, y_values)

### Berechnung der Punkte für einen Poincaré-Plot

Für einen Startpunkt am Ort $(x|0)$ und eine anfängliche Bewegungsrichtung, die den Winkel $\alpha$ mit der Horizontalen bildet, werden `n_out` Punkte für einen Poincaré-Plot berechnet. Hierzu werden die Kollisionen mit dem Berandungsstück 2, also der schrägen Wand im symmetriereduzierten Modell, herangezogen.

In [ ]:
def poincare_points(x, alpha, n_out):
    y = 0
    v_x, v_y = (cos(alpha), sin(alpha))
    eps = 1e-8
    s_values = []
    alpha_values = []
    contacts = stroboscopic_map(SINAI, x, y, v_x, v_y, eps)
    n = 0
    while n < n_out:
        n_boundary, x, y, v_x, v_y = next(contacts)
        if n_boundary == 2:
            n = n+1
            s_values.append((2*hypot(x, y)-1)/(2*sqrt(2)-1))
            alpha_values.append(atan2(v_y, v_x)+pi/4)
    return s_values, alpha_values

### Graphische Darstellung des Poincaré-Plots

Für einen ausgewählten Startpunkt auf der horizontalen Berandung bei $y=0$ wird ein Poincaré-Plot dargestellt.

In [ ]:
@interact(**get_widgets())
def plot_result_poincare(x, alpha, n_out_poincare):
    s_values, alpha_values = poincare_points(
        x, alpha, n_out_poincare)

    fig, ax = plt.subplots()
    ax.plot(s_values, alpha_values, linestyle="None",
            marker="o", markersize=2)
    ax.set_xlabel("$s$")
    ax.set_ylabel(r"$\alpha$")